In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [2]:
import sys
import numpy as np
import os
import binascii
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col

In [9]:
directory = os.listdir('/Users/Bayernlad/Downloads/cookbook_text')
dataArr = [None] * (len(directory)-72)

In [10]:
for i in range(len(directory)-72):
    f = open('/Users/Bayernlad/Downloads/cookbook_text/' + str(directory[i]), 'r')
    words = f.read().split()
    shingles = []
    
    j = 0
    while j < len(words):
        if words[j].isalnum() is False:
            del words[j]
        else:
            j += 1
            
    for j in range(len(words) - 2):
        shingle = words[j] + " " + words[j + 1] + " " + words[j + 2]
        crc = binascii.crc32(shingle) & 0xffffffff
        shingles.append(crc)
        
    f.close()
    
    for j in range(len(shingles)):
        shingles[j] = float(shingles[j])
        
    shingles.sort()
    j = 0
    while j < len(shingles):
        if (shingles[j] >= 2 ** 31) or (shingles[j] < 0):
            del shingles[j]
        else:
            j += 1
            
    data = [(i, Vectors.dense(np.array(shingles)))]
    df = spark.createDataFrame(data, ["id", "features"])
    dataArr[i] = df

In [12]:
mh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=5)
for i in range(len(directory)-72):
    model = mh.fit(dataArr[i])
    for j in range(i + 1, len(directory)-72):
        model.approxSimilarityJoin(dataArr[i], dataArr[j], float(2 ** 64), distCol="JaccardDistance")\
        .select(col("datasetA.id").alias(directory[i]),
                col("datasetB.id").alias(directory[j]),
                col("JaccardDistance")).show()

+--------+--------+------------------+
|henr.txt|cclu.txt|   JaccardDistance|
+--------+--------+------------------+
|       0|       1|0.8367974753953209|
+--------+--------+------------------+

+--------+--------+------------------+
|henr.txt|beec.txt|   JaccardDistance|
+--------+--------+------------------+
|       0|       2|0.5060220448843835|
+--------+--------+------------------+

+--------+--------+------------------+
|henr.txt|hous.txt|   JaccardDistance|
+--------+--------+------------------+
|       0|       3|0.7568138217565528|
+--------+--------+------------------+

+--------+--------+------------------+
|cclu.txt|beec.txt|   JaccardDistance|
+--------+--------+------------------+
|       1|       2|0.6696157735085946|
+--------+--------+------------------+

+--------+--------+------------------+
|cclu.txt|hous.txt|   JaccardDistance|
+--------+--------+------------------+
|       1|       3|0.3288988470335621|
+--------+--------+------------------+

+--------+--------+-